In [1]:
import pychomp

# help and docstrings

In addition to looking at the source code, the python `help` command gives information.
The information it gives could be improved by introducing "docstrings" into the python wrapping code. For a listing of package/module contents you can also use `dir`. Examples:

    dir(pychomp)
    help(pychomp)
    
Notice there is a subpackage `pychomp._chomp`. This is the subpackage for the so-called _extension module_ which is the C++ code that has a python wrapper around it. The technology I am using for the python wrapping is `pybind11`.

The classes and functions living in `pychomp._chomp` are imported into the top level so you don't have to write `pychomp._chomp.XXX` but rather simply `pychomp.XXX`. The annoying thing, however, is that the various things don't show up when you type `help(pychomp)`, and you'd have to type `help(pychomp._chomp)`. Perhaps there is a better way of packaging the python code and extension module together. You can poke around with the following commands:

    dir(pychomp._chomp)
    help(pychomp._chomp)
    help(pychomp.SimplicialComplex)
    help(pychomp.DualComplex)
    

# Complexes in pychomp

For the present discussion, by _chain complex_ we specifically mean a chain complex over finite dimensional vector spaces (i.e. fields rather than rings). In `pychomp`, the field is currently $\mathbb{F}_2$. 

A _cell complex_ is a chain complex $C$ equipped with a distinguished basis for each $C_n$ such that at most finitely many of the $C_n$ are nontrivial. We demand the dimensional grading of the $C_n$ are non-negative integers. We call the maximum grade $n$ for which $C_n \neq 0$ the _dimension of the complex_.

The complexes of `pychomp` are cell complexes, though we may regard them as chain complexes when dealing with them in a way that is insensitive to the distinguished bases. 

The word _dimension_ is unfortunately rather overloaded in the present context, as it can refer to three distinct things:

1. (grade-dimension) The grading parameter $n$ of a chain complex $C = (C_n, d_n)$
2. (space-dimension) The dimension of a vector space
3. (complex-dimension) The dimension of the complex, i.e. the maximum $n$ for which $C_n \neq 0$


Given an instance `C` of (a subclass of) the `pychomp.Complex` class, we have the following methods:

* `C.dimension()` -- returns the complex-dimension of the complex `C`
* `C.count()` -- returns a list of space-dimensions $\dim C_n$ for $n = 0, \cdots,$`C.dimension()`

Note: It is named `count` is named thus as it counts the number of cells at each grade-dimension.

# Building an example Simplicial Complex

A simplicial complex is built from a list of (maximal) simplices. It is OK to include non-maximal simplices; they are ignored as they are built anyway.

In [2]:
simplices = [ [1,2,3], [0,2,3], [0,1,3], [0,1,2], [4,5,6,7]]

In [3]:
C = pychomp.SimplicialComplex(simplices)

In [4]:
print("C.dimension() =", C.dimension())
print("C.count() =", C.count())

C.dimension() = 3
C.count() = [8, 12, 8, 1]


# Homology 

The homology modules $(H_n(C))$ of a chain complex $C$ are in fact vector spaces and may be equipped with zero differentials $0_n : H_{n+1}(C) \to H_n(C)$ to give another chain complex which we call $H_*(C)$. We call this the _homology complex_.

**Remark.** In fact, in this setting we may realize $H_*$ as an endofunctor on the category of chain complexes. This endofunctor is idempotent up to isomorphism, i.e. $H_* ( H_* ( C ) ) \simeq H_*(C)$. 

**Theorem.** Any chain complex with zero differentials that is chain equivalent to $C$ is isomorphic to $H_*(C)$.

_Proof._ This is straightforward from: (1) chain equivalence is an equivalence relation, (2) homology is an invariant of chain equivalence classes, and (3) chain complexes $C$ and $D$ with zero differentials are chain equivalent iff they are isomorphic (i.e. iff $\dim C_n = \dim D_n$ for each $n$). $\square$

This theorem informs the computational strategy of `pychomp` which uses repeated rounds of reduction (via discrete Morse theory) to give a sequence of chain equivalences leading to a fully reduced complex (i.e. a complex with zero differentials). This is implemented as `pychomp.Homology`.

To summarize, let `C` be a chain complex. Then `pychomp.Homology(C)` yields a chain complex with zero differentials that is chain equivalent to `C`, which is isomorphic to (and thus might as well be identified with) with the homology complex $H_*(C)$.




In [5]:
HC = pychomp.Homology(C)
C_betti_numbers = HC.count()
print(C_betti_numbers)

[2, 0, 1, 0]


# Dual Complex

`pychomp` can construct a _dual complex_ from another complex.

Given a complex $C = (C_n, d_n)$ with complex-dimension $d$, we define the dual complex $(D_n, e_n)$ via

* $D_{k} := C_{d-k}$
* $e_{k} := d^T_{d-k+1}$

In [6]:
simplices = [ [1,2,3], [0,2,3], [0,1,3], [0,1,2], [4,5,6,7]]
C = pychomp.SimplicialComplex(simplices)
D = pychomp.DualComplex(C)

In [7]:
print("C.dimension() =", C.dimension())
print("C.count() =", C.count())
print("C betti numbers = ", pychomp.Homology(C).count())
print("D.dimension() =", D.dimension())
print("D.count() =", D.count())
print("D betti numbers = ", pychomp.Homology(D).count())

C.dimension() = 3
C.count() = [8, 12, 8, 1]
C betti numbers =  [2, 0, 1, 0]
D.dimension() = 3
D.count() = [1, 8, 12, 8]
D betti numbers =  [0, 1, 0, 2]
